<a href="https://colab.research.google.com/github/gaallmin/Prediction-of-Agricultural-Product-prices/blob/yeji/%EA%B2%B0%EC%B8%A1%EC%B9%98_%EB%8C%80%EC%B2%B4_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive/')
%cd /content/drive/MyDrive/Dacon

Mounted at /content/drive/
/content/drive/MyDrive/Dacon


In [2]:
import pandas as pd

train_file_path = './train/train.csv'
market_file_path = './train/meta/TRAIN_전국도매_2018-2021.csv'
sanjae_file_path = './train/meta/TRAIN_산지공판장_2018-2021.csv'

train_df = pd.read_csv(train_file_path)
market_df = pd.read_csv(market_file_path)
sanjae_df = pd.read_csv(sanjae_file_path)

In [9]:
'''
import numpy as np

train_df['year'] = train_df['시점'].str[:4].astype(int)
train_df['month'] = train_df['시점'].str[4:6].astype(int)
train_df['순'] = train_df['시점'].str[6:8]
train_df = train_df[['year', 'month', '순'] + [col for col in train_df.columns if col not in ['year', 'month', '순']]]
'''

In [4]:
'''
import pandas as pd

# 품종 쌍을 설정 (화건-햇산화건, 양건-햇산양건)
product_pairs = [('화건', '햇산화건'), ('양건', '햇산양건')]

# 가격 컬럼 리스트
price_columns = ['평균가격(원)', '평년 평균가격(원)']

# 평균 비율을 계산하는 함수
def calculate_mean_ratio(df1, df2, price_col):
    # 시점 기준으로 병합
    merged_df = pd.merge(df1[['시점', price_col, '등급']], df2[['시점', price_col, '등급']],
                         on=['시점', '등급'], suffixes=('_1', '_2'), how='inner')

    # 두 품종의 가격이 모두 0이 아닌 경우만 사용
    valid_rows = merged_df[(merged_df[f'{price_col}_1'] != 0) & (merged_df[f'{price_col}_2'] != 0)]

    # 비율 계산 및 평균 비율 구하기
    valid_rows['비율'] = valid_rows[f'{price_col}_1'] / valid_rows[f'{price_col}_2']
    mean_ratio = valid_rows['비율'].mean()

    return mean_ratio

# 0인 값을 대체하는 함수
def replace_zero_in_train(row, mean_ratio, product_1, product_2, price_col):
    if row['품종명'] == product_1:
        # 같은 시점, 등급의 다른 품종 가격 가져오기
        matching_row = train_df[(train_df['시점'] == row['시점']) & (train_df['품종명'] == product_2) & (train_df['등급'] == row['등급'])]
        if not matching_row.empty:
            other_price = matching_row[price_col].values[0]
            if row[price_col] == 0 and other_price != 0:
                row[price_col] = other_price * mean_ratio
    elif row['품종명'] == product_2:
        # 같은 시점, 등급의 다른 품종 가격 가져오기
        matching_row = train_df[(train_df['시점'] == row['시점']) & (train_df['품종명'] == product_1) & (train_df['등급'] == row['등급'])]
        if not matching_row.empty:
            other_price = matching_row[price_col].values[0]
            if row[price_col] == 0 and other_price != 0:
                row[price_col] = other_price / mean_ratio
    return row

# 0 대체 로직 적용 함수
def apply_replacement(product_1, product_2, price_col, group_by_columns):
    for group_values in train_df[group_by_columns].drop_duplicates().itertuples(index=False):
        # group_by_columns에 따른 필터링
        df1 = train_df[(train_df['품종명'] == product_1) & (train_df[list(group_by_columns)] == group_values).all(axis=1)]
        df2 = train_df[(train_df['품종명'] == product_2) & (train_df[list(group_by_columns)] == group_values).all(axis=1)]

        # 평균 비율 계산
        mean_ratio = calculate_mean_ratio(df1, df2, price_col)
        print(f"{product_1}-{product_2}의 {price_col}에 대한 평균 비율 {group_by_columns}: {group_values} : {mean_ratio}")

        # 0인 값을 평균 비율로 대체
        train_df.update(train_df.apply(replace_zero_in_train, axis=1, mean_ratio=mean_ratio,
                                       product_1=product_1, product_2=product_2, price_col=price_col))

# 순차적으로 대체 작업 수행 - 'month'와 '순'은 TEST에서 알 수 없어서 일단 제외.
for product_1, product_2 in product_pairs:
    for price_col in price_columns:
        # Step 1: 'month'와 '순' 기준으로 대체
        apply_replacement(product_1, product_2, price_col, ['month', '순'])

        # Step 2: 'month' 기준으로 대체
        apply_replacement(product_1, product_2, price_col, ['month'])

        # Step 3: 'year' 기준으로 대체
        apply_replacement(product_1, product_2, price_col, ['year'])

        # Step 4: 전체 기준으로 대체 (기본 전체 적용)
        apply_replacement(product_1, product_2, price_col, [])

# 결과 확인
train_df.head()
'''

,year,month,순,시점,품목명,품종명,거래단위,등급,평년 평균가격(원),평균가격(원)
0,2018,1,상순,201801상순,건고추,화건,30 kg,상품,381666.666667,590000.0
1,2018,1,중순,201801중순,건고추,화건,30 kg,상품,380809.666667,590000.0
2,2018,1,하순,201801하순,건고추,화건,30 kg,상품,380000.000000,590000.0
3,2018,2,상순,201802상순,건고추,화건,30 kg,상품,380000.000000,590000.0
4,2018,2,중순,201802중순,건고추,화건,30 kg,상품,376666.666667,590000.0


In [6]:
'''
import pandas as pd

# 가격 컬럼 리스트
price_columns = ['평균가격(원)', '평년 평균가격(원)']

# 가장 적은 0 값이 있는 등급을 찾는 함수
def get_min_zero_grade(df, price_col):
    return df.groupby('등급').apply(lambda x: (x[price_col] == 0).sum()).idxmin()

# 평균 비율을 계산하는 함수
def calculate_mean_ratio(df1, df2, price_col):
    merged_df = pd.merge(df1[['시점', price_col, '품종명']], df2[['시점', price_col, '품종명']],
                         on=['시점', '품종명'], suffixes=('_1', '_2'), how='inner')

    valid_rows = merged_df[(merged_df[f'{price_col}_1'] != 0) & (merged_df[f'{price_col}_2'] != 0)]
    valid_rows['비율'] = valid_rows[f'{price_col}_1'] / valid_rows[f'{price_col}_2']

    return valid_rows['비율'].mean()

# 0인 값을 대체하는 함수
def replace_zero_in_train(row, mean_ratio, grade_1, grade_2, price_col):
    if row['등급'] == grade_1:
        matching_row = train_df[(train_df['시점'] == row['시점']) & (train_df['등급'] == grade_2) & (train_df['품종명'] == row['품종명'])]
        if not matching_row.empty:
            other_price = matching_row[price_col].values[0]
            if row[price_col] == 0 and other_price != 0:
                row[price_col] = other_price * mean_ratio
    elif row['등급'] == grade_2:
        matching_row = train_df[(train_df['시점'] == row['시점']) & (train_df['등급'] == grade_1) & (train_df['품종명'] == row['품종명'])]
        if not matching_row.empty:
            other_price = matching_row[price_col].values[0]
            if row[price_col] == 0 and other_price != 0:
                row[price_col] = other_price / mean_ratio
    return row

# 0 대체 로직을 적용하는 함수
def apply_replacement(product_1, price_col, grade_pairs, group_by_columns):
    for group_values in train_df[group_by_columns].drop_duplicates().itertuples(index=False):
        for grade_1, grade_2 in grade_pairs:
            # 해당 등급과 조건에 맞는 데이터 필터링
            df1 = train_df[(train_df['등급'] == grade_1) & (train_df[list(group_by_columns)] == group_values).all(axis=1) & (train_df['품종명'] == product_1)]
            df2 = train_df[(train_df['등급'] == grade_2) & (train_df[list(group_by_columns)] == group_values).all(axis=1) & (train_df['품종명'] == product_1)]

            # 평균 비율 계산
            mean_ratio = calculate_mean_ratio(df1, df2, price_col)
            print(f"{product_1}의 {grade_1}-{grade_2} {price_col} 평균 비율 {group_by_columns}: {group_values} : {mean_ratio}")

            # 0인 값을 평균 비율로 대체
            train_df.update(train_df.apply(replace_zero_in_train, axis=1, mean_ratio=mean_ratio,
                                           grade_1=grade_1, grade_2=grade_2, price_col=price_col))

# 각 품종별로 가장 적은 0 값이 있는 등급을 기준으로 나머지 등급과 쌍을 이루는 함수
def create_grade_pairs_by_product(df, product, price_col):
    # 품종별 데이터 필터링
    product_df = df[df['품종명'] == product]

    # 기준 등급 (가장 적은 0 값이 있는 등급)
    base_grade = get_min_zero_grade(product_df, price_col)

    # 나머지 등급과 기준 등급으로 쌍 생성
    other_grades = product_df['등급'].unique().tolist()
    other_grades.remove(base_grade)

    return [(base_grade, grade) for grade in other_grades]

# 순차적으로 대체 작업 수행
for product in train_df['품종명'].unique():
    for price_col in price_columns:
        # 품종별 등급 쌍 생성
        grade_pairs = create_grade_pairs_by_product(train_df, product, price_col)

        for grade_pair in grade_pairs:

          # Step 1: 'month'와 '순' 기준으로 대체
          # apply_replacement(product, price_col, grade_pairs, ['month', '순'])

          # Step 2: 'month' 기준으로 대체
          # apply_replacement(product, price_col, grade_pairs, ['month'])

          # Step 3: 'year' 기준으로 대체
          # apply_replacement(product, price_col, grade_pairs, ['year'])

          # Step 4: 전체 데이터 기준으로 대체
          apply_replacement(product, price_col, grade_pair, [])

# 결과 확인
train_df.head()
'''

<ipython-input-6-7b51b4fada31>:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('등급').apply(lambda x: (x[price_col] == 0).sum()).idxmin()
<ipython-input-6-7b51b4fada31>:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('등급').apply(lambda x: (x[price_col] == 0).sum()).idxmin()
<ipython-input-6-7b51b4fada31>:8: DeprecationWarning: DataFrameGroupBy.apply operated on the group

,year,month,순,시점,품목명,품종명,거래단위,등급,평년 평균가격(원),평균가격(원)
0,2018,1,상순,201801상순,건고추,화건,30 kg,상품,381666.666667,590000.0
1,2018,1,중순,201801중순,건고추,화건,30 kg,상품,380809.666667,590000.0
2,2018,1,하순,201801하순,건고추,화건,30 kg,상품,380000.000000,590000.0
3,2018,2,상순,201802상순,건고추,화건,30 kg,상품,380000.000000,590000.0
4,2018,2,중순,201802중순,건고추,화건,30 kg,상품,376666.666667,590000.0


In [9]:
import pandas as pd

# 품종 쌍과 거래단위를 설정
product_pairs = [
    ('양파', '20키로', '양파', '10키로'),
    ('양파', '12키로', '양파(햇)', '12키로'),
    ('양파', '15키로', '양파(햇)', '15키로'),
    ('양파', '20키로', '양파(햇)', '20키로'),
    ('양파', '15키로', '저장양파', '15키로'),
    ('양파', '12키로', '저장양파', '12키로'),
    ('양파', '20키로', '저장양파', '20키로'),
    ('양파', '12키로', '조생양파', '12키로'),
    ('양파', '15키로', '조생양파', '15키로'),
    ('감자 수미', '20키로상자', '감자', '20키로상자'),
    ('감자 수미', '20키로상자', '감자 대지', '20키로상자'),
    ('감자 수미', '20키로상자', '감자 두백', '20키로상자'),
    ('감자 수미', '20키로상자', '감자 수미(저장)', '20키로상자'),
    ('감자 수미', '20키로상자', '감자 수미(햇)', '20키로상자'),
    ('감자 수미', '20키로상자', '감자 수입', '23키로상자'),
    ('감자 수미', '20키로상자', '감자 조풍', '20키로상자'),
    ('감자 수미', '20키로상자', '홍감자', '10키로상자'),
    ('화건', '30 kg', '햇산화건', '30 kg'),
    ('양건', '30 kg', '햇산양건', '30 kg'),
    ('실파', '20키로', '쪽파', '10키로상자'),
    ('쪽파', '10키로상자', '대파 수입', '10키로묶음'),
    ('쪽파', '10키로상자', '대파(일반)', '10키로묶음'),
    ('쪽파', '10키로상자', '대파(일반)', '1키로단'),
    ('무', '20키로상자', '무', '18키로상자(비닐포)'),
    ('무', '20키로상자', '다발무', '5톤트럭'),
    ('신고', '10 개', '원황', '10 개'),
    ('봄동배추', '15키로상자', '절임배추', '20키로상자'),
    ('쌈배추', '8키로상자', '쌈배추', '1키로상자'),
    ('얼갈이배추', '4키로상자', '얼갈이배추', '8키로상자'),
    ('후지', '10 개', '쓰가루', '10 개')
]

# 가격 컬럼 리스트
price_columns = ['평균가격(원)', '평년 평균가격(원)']

# 평균 비율을 계산하는 함수 (거래단위를 포함)
def calculate_mean_ratio(df1, df2, price_col):
    # '시점', '등급', '거래단위'를 기준으로 inner join
    merged_df = pd.merge(df1[['시점', price_col, '등급', '거래단위']],
                         df2[['시점', price_col, '등급', '거래단위']],
                         on=['시점', '등급', '거래단위'], suffixes=('_1', '_2'), how='inner')

    # 두 품종의 가격이 모두 0이 아닌 경우만 사용
    valid_rows = merged_df[(merged_df[f'{price_col}_1'] != 0) & (merged_df[f'{price_col}_2'] != 0)]

    # 비율 계산 및 평균 비율 구하기
    valid_rows['비율'] = valid_rows[f'{price_col}_1'] / valid_rows[f'{price_col}_2']
    mean_ratio = valid_rows['비율'].mean()

    return mean_ratio

# 0인 값을 대체하는 함수 (평균 비율을 사용)
def replace_zero_in_train(row, mean_ratio, product_1, unit_1, product_2, unit_2, price_col):
    if row['품종명'] == product_1 and row['거래단위'] == unit_1:
        # 같은 시점, 등급, 거래단위의 다른 품종 가격 가져오기
        matching_row = train_df[(train_df['시점'] == row['시점']) &
                                (train_df['품종명'] == product_2) &
                                (train_df['등급'] == row['등급']) &
                                (train_df['거래단위'] == unit_2)]
        if not matching_row.empty:
            other_price = matching_row[price_col].values[0]
            if row[price_col] == 0 and other_price != 0:
                row[price_col] = other_price * mean_ratio
    elif row['품종명'] == product_2 and row['거래단위'] == unit_2:
        # 같은 시점, 등급, 거래단위의 첫 번째 품종 가격 가져오기
        matching_row = train_df[(train_df['시점'] == row['시점']) &
                                (train_df['품종명'] == product_1) &
                                (train_df['등급'] == row['등급']) &
                                (train_df['거래단위'] == unit_1)]
        if not matching_row.empty:
            other_price = matching_row[price_col].values[0]
            if row[price_col] == 0 and other_price != 0:
                row[price_col] = other_price / mean_ratio
    return row

# 각 품종 쌍과 가격 컬럼에 대해 처리 적용
for product_1, unit_1, product_2, unit_2 in product_pairs:
    for price_col in price_columns:
        # 해당 품종과 거래단위 필터링
        df1 = train_df[(train_df['품종명'] == product_1) & (train_df['거래단위'] == unit_1)]
        df2 = train_df[(train_df['품종명'] == product_2) & (train_df['거래단위'] == unit_2)]

        # 평균 비율 계산
        mean_ratio = calculate_mean_ratio(df1, df2, price_col)
        print(f"{product_1}({unit_1}) - {product_2}({unit_2})의 {price_col}에 대한 평균 비율: {mean_ratio}")

        # 0인 값을 평균 비율로 대체
        train_df = train_df.apply(replace_zero_in_train, axis=1, mean_ratio=mean_ratio,
                                  product_1=product_1, unit_1=unit_1,
                                  product_2=product_2, unit_2=unit_2,
                                  price_col=price_col)


양파(20키로) - 양파(10키로)의 평균가격(원)에 대한 평균 비율: nan
양파(20키로) - 양파(10키로)의 평년 평균가격(원)에 대한 평균 비율: nan
양파(12키로) - 양파(햇)(12키로)의 평균가격(원)에 대한 평균 비율: 1.0126329222765227
양파(12키로) - 양파(햇)(12키로)의 평년 평균가격(원)에 대한 평균 비율: nan
양파(15키로) - 양파(햇)(15키로)의 평균가격(원)에 대한 평균 비율: 1.0011656821945343
양파(15키로) - 양파(햇)(15키로)의 평년 평균가격(원)에 대한 평균 비율: nan


<ipython-input-9-335025cab461>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_rows['비율'] = valid_rows[f'{price_col}_1'] / valid_rows[f'{price_col}_2']


양파(20키로) - 양파(햇)(20키로)의 평균가격(원)에 대한 평균 비율: 1.0163299068965257


<ipython-input-9-335025cab461>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_rows['비율'] = valid_rows[f'{price_col}_1'] / valid_rows[f'{price_col}_2']


양파(20키로) - 양파(햇)(20키로)의 평년 평균가격(원)에 대한 평균 비율: nan


...

신고(10 개) - 원황(10 개)의 평년 평균가격(원)에 대한 평균 비율: 1.2324696351273512
봄동배추(15키로상자) - 절임배추(20키로상자)의 평균가격(원)에 대한 평균 비율: nan
봄동배추(15키로상자) - 절임배추(20키로상자)의 평년 평균가격(원)에 대한 평균 비율: nan
쌈배추(8키로상자) - 쌈배추(1키로상자)의 평균가격(원)에 대한 평균 비율: nan
쌈배추(8키로상자) - 쌈배추(1키로상자)의 평년 평균가격(원)에 대한 평균 비율: nan
얼갈이배추(4키로상자) - 얼갈이배추(8키로상자)의 평균가격(원)에 대한 평균 비율: nan
얼갈이배추(4키로상자) - 얼갈이배추(8키로상자)의 평년 평균가격(원)에 대한 평균 비율: nan


<ipython-input-9-335025cab461>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_rows['비율'] = valid_rows[f'{price_col}_1'] / valid_rows[f'{price_col}_2']


후지(10 개) - 쓰가루(10 개)의 평균가격(원)에 대한 평균 비율: 1.402595682437155


<ipython-input-9-335025cab461>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_rows['비율'] = valid_rows[f'{price_col}_1'] / valid_rows[f'{price_col}_2']


후지(10 개) - 쓰가루(10 개)의 평년 평균가격(원)에 대한 평균 비율: 1.5405378902819928


In [10]:
import pandas as pd
import re

# 품종 쌍과 거래단위 설정
product_pairs = [
    ('다발무', '5톤트럭', '다발무', '10키로'),
    ('다발무', '5톤트럭', '다발무', '5000키로'),
    ('다발무', '5톤트럭', '다발무', '1000키로'),
    ('다발무', '5톤트럭', '다발무', '10키로'),
    ('다발무', '5톤트럭', '다발무', '8톤트럭')
]

# 가격 및 거래단위 컬럼 리스트
price_columns = ['평균가격(원)', '평년 평균가격(원)']
unit_column = '거래단위_숫자'

# '거래단위'에서 'kg'을 제외한 숫자만 추출하여 새로운 컬럼 '거래단위_숫자' 생성
train_df['거래단위_숫자'] = train_df['거래단위'].apply(lambda x: float(re.sub(r'[^0-9.]', ' ', x)))

# '거래단위'에서 '5톤트럭'과 '8톤트럭' 처리
def process_truck_units(unit):
    if '5톤트럭' in unit:
        return 5000
    elif '8톤트럭' in unit:
        return 8000
    else:
        return float(re.sub(r'[^0-9.]', '', unit))  # 그 외의 경우 숫자만 추출

train_df['거래단위_숫자'] = train_df['거래단위'].apply(process_truck_units)

# 0인 값을 대체하는 함수
def replace_zero_with_unit_price(row, product_1, unit_1, product_2, unit_2, price_col):
    if row['품종명'] == product_1 and row['거래단위'] == unit_1:
        # 같은 시점, 등급의 다른 품종과 거래단위의 가격 가져오기
        matching_row = train_df[(train_df['시점'] == row['시점']) &
                                (train_df['품종명'] == product_2) &
                                (train_df['등급'] == row['등급']) &
                                (train_df['거래단위'] == unit_2)]
        if not matching_row.empty:
            other_price = matching_row[price_col].values[0]
            if row[price_col] == 0 and other_price != 0:
                # 첫 번째 품종의 가격이 0이면 두 번째 품종의 가격에 거래단위_숫자를 곱해 단위가격으로 대체
                row[price_col] = other_price * row['거래단위_숫자'] / matching_row['거래단위_숫자'].values[0]
    elif row['품종명'] == product_2 and row['거래단위'] == unit_2:
        # 같은 시점, 등급의 첫 번째 품종과 거래단위의 가격 가져오기
        matching_row = train_df[(train_df['시점'] == row['시점']) &
                                (train_df['품종명'] == product_1) &
                                (train_df['등급'] == row['등급']) &
                                (train_df['거래단위'] == unit_1)]
        if not matching_row.empty:
            other_price = matching_row[price_col].values[0]
            if row[price_col] == 0 and other_price != 0:
                # 두 번째 품종의 가격이 0이면 첫 번째 품종의 단위가격을 대체
                row[price_col] = (other_price / matching_row['거래단위_숫자'].values[0]) * row['거래단위_숫자']
    return row

# 각 품종 쌍과 가격 컬럼에 대해 처리 적용
for product_1, unit_1, product_2, unit_2 in product_pairs:
    for price_col in price_columns:
        # 0인 값을 단위가격으로 대체
        train_df = train_df.apply(replace_zero_with_unit_price, axis=1,
                                  product_1=product_1, unit_1=unit_1,
                                  product_2=product_2, unit_2=unit_2, price_col=price_col)


In [12]:
train_df['평균가격(원)'] = train_df['평균가격(원)'].fillna(0)
train_df['평균가격(원)'] = train_df['평균가격(원)'].apply(lambda x: 0 if x == 0 or pd.isna(x) else x)

train_df['평년 평균가격(원)'] = train_df['평년 평균가격(원)'].fillna(0)
train_df['평년 평균가격(원)'] = train_df['평년 평균가격(원)'].apply(lambda x: 0 if x == 0 or pd.isna(x) else x)

train_df = train_df.drop(['거래단위_숫자'], axis = 1)

In [23]:
# 수정된 데이터를 새로운 CSV 파일로 저장 (UTF-8-SIG 인코딩 사용)
output_file_path = './train_v1.csv'
train_df.to_csv(output_file_path, index=False, encoding='utf-8-sig')

In [20]:
import pandas as pd

# 데이터 불러오기
trainv1_df = pd.read_csv('./train_v1.csv')

# 필터링할 조건 설정 (행별로 조건을 각각 명시)
condition_1 = (
    (trainv1_df['품목명'] == '건고추') &
    (trainv1_df['품종명'] == '화건') &
    (trainv1_df['거래단위'] == '30 kg') &
    (trainv1_df['등급'] == '상품')
)

condition_2 = (
    (trainv1_df['품목명'] == '사과') &
    (trainv1_df['품종명'].isin(['홍로', '후지'])) &
    (trainv1_df['거래단위'] == '10 개') &
    (trainv1_df['등급'] == '상품')
)

condition_3 = (
    (trainv1_df['품목명'] == '감자') &
    (trainv1_df['품종명'] == '감자 수미') &
    (trainv1_df['거래단위'] == '20키로상자') &
    (trainv1_df['등급'] == '상')
)

condition_4 = (
    (trainv1_df['품목명'] == '배') &
    (trainv1_df['품종명'] == '신고') &
    (trainv1_df['거래단위'] == '10 개') &
    (trainv1_df['등급'] == '상품')
)

condition_5 = (
    (trainv1_df['품목명'] == '깐마늘(국산)') &
    (trainv1_df['품종명'] == '깐마늘(국산)') &
    (trainv1_df['거래단위'] == '20 kg') &
    (trainv1_df['등급'] == '상품')
)

condition_6 = (
    (trainv1_df['품목명'] == '무') &
    (trainv1_df['품종명'] == '무') &
    (trainv1_df['거래단위'] == '20키로상자') &
    (trainv1_df['등급'] == '상품')
)

condition_7 = (
    (trainv1_df['품목명'] == '상추') &
    (trainv1_df['품종명'] == '청') &
    (trainv1_df['거래단위'] == '100 g') &
    (trainv1_df['등급'] == '상품')
)

condition_8 = (
    (trainv1_df['품목명'] == '배추') &
    (trainv1_df['품종명'] == '배추') &
    (trainv1_df['거래단위'] == '10키로망대') &
    (trainv1_df['등급'] == '상')
)

condition_9 = (
    (trainv1_df['품목명'] == '양파') &
    (trainv1_df['품종명'] == '양파') &
    (trainv1_df['거래단위'] == '1키로') &
    (trainv1_df['등급'] == '상')
)

condition_10 = (
    (trainv1_df['품목명'] == '대파') &
    (trainv1_df['품종명'] == '대파(일반)') &
    (trainv1_df['거래단위'] == '1키로단') &
    (trainv1_df['등급'] == '상')
)

# 모든 조건을 합치기 (OR로 연결)
final_condition = condition_1 | condition_2 | condition_3 | condition_4 | condition_5 | condition_6 | condition_7 | condition_8 | condition_9 | condition_10

# 조건에 맞는 데이터 추출
filtered_df = trainv1_df[final_condition]
filtered_df = filtered_df.drop(['거래단위_숫자'], axis = 1)

In [21]:
# 수정된 데이터를 새로운 CSV 파일로 저장 (UTF-8-SIG 인코딩 사용)
output_file_path = './train_v1_s.csv'
filtered_df.to_csv(output_file_path, index=False, encoding='utf-8-sig')